# Introduction to naming in Ruby
## Local Variables

In [1]:
a = 4
puts defined?(a) ? a : 'a is not defined'   # this is a fancy way to avoid NameErrors

def local_var
  b = 5
end
puts defined?(b) ? b : 'b is not defined'

c = 6
def top_level_locality
  puts defined?(c) ? c : 'c is not defined'
end
top_level_locality

4
b is not defined
c is not defined


It is interesting to note that though we declared the variable c in the top-level, it was not defined in the method `top_level_locality`.  This is because the top-level is a local scope in Ruby. We declare global variables in Ruby using the `$` characeter. More on that later.

### Interesting cases
We can overcome the encapsulation displayed above using certain closure definitions... In the function `inner_scope`, the value of the variable word is `bar`, yet each of the outputs prints `foo` becasuse of closure definitions. Read more about that [here](https://ruby-doc.org/core-2.5.1/Binding.html).

In [2]:
word = "foo"
define_method :x do
  puts word
end

y = Proc.new {puts word}

z = lambda {puts word}
  
def inner_scope(proc_obj, lambda_obj)
  word = "bar"
  x
  proc_obj.call
  lambda_obj.call
end
  
inner_scope(y, z)

foo
foo
foo


While it is poor practice to do so, it is possible to assign the same name to both a method and a local variable. In that case, the variable will preceed the method. In the example below, the puts statements will output a 4 if they refer to the variable a and a 5 if they refer to the method.

In [3]:
def a
    5
end
puts a
# If you really need to get around this method/variable naming limitation, you can use the send keyword...
puts send :a

4
5


## Global Variables
Below is an example of how to declare and use global variables. Placing the variable in all caps will accomplish the same thing.

In [4]:
$c = 6
def global_ex
  puts defined?($c) ? $c : '$c is not defined'
end
global_ex

D = 7
def const_ex
  puts defined?(D) ? D : 'D is not defined'
end
const_ex

6
7


## Class and Instance Variables
Class variables are denoted by `@@` while instance variables are denoted by `@`. There are some rather interesting identifier linkages formed by Ruby with respect to an object's methods. More on that in a bit. Below, notice how even though we set `id` to `false` both singleton-instances of the Customer class have their unique id's.

In [5]:
class Customer
  @@no_of_customers = 0

  def initialize(id, name, addr)
    @id = id
    id = 'false'       # this is different than setting the instance variable
    @name = name
    @addr = addr
    @@no_of_customers += 1
  end
  
  def print_id
    puts @id
  end
  
  class << self
    def print_num_cust
      puts @@no_of_customers
    end
  end
end

cust1 = Customer.new('9999', 'Caleb Bitting', 'Mary Low 311')
cust2 = Customer.new('8888', 'John Connors', 'Mary Low 312')

cust1.print_id
cust2.print_id

9999
8888


Notice that weird `class << self` section? That marks a shift to defining class methods. Notice what happens when we try to call the method `print_num_cust` on an instance and on the Class itself.

In [6]:
begin
  cust1.print_num_cust
rescue
  puts 'An error occured when attempting to print the number of customers from an object'
end

Customer.print_num_cust

An error occured when attempting to print the number of customers from an object
2


## A brief discussion about Modules
A module behaves much the same as a class but modules cannot generate instances. See the examples below for nearly identical funcitonality as described when talking about classes.

In [7]:
module Car
  def self.description
    "A vehicle of transportation"
  end
  def engine
    "vroom"
  end
end

# call the Module method on the Module
puts Car.description

# try to call a method defined in the Module on the Module itself
begin
  Car.engine
rescue
  puts "An error occured when trying to call the method engine on the Module Car"
end

A vehicle of transportation
An error occured when trying to call the method engine on the Module Car


In [8]:
# Create a class that uses the module defined above
class RaceCar
  include Car
end

f1 = RaceCar.new
# call the method defined in the above module
puts f1.engine

# try to call the Module method defined above
begin
  puts f1.description
rescue
  puts "An error occured when trying to call the method description on the instance of RaceCar named f1"
end

vroom
An error occured when trying to call the method description on the instance of RaceCar named f1


A final note on Modules: they are the best way to implement the an Enum from Java. See below

In [9]:
module Foo
  BAR = 10
  BAZ = 100
  BIN = 1000
end

puts Foo::BAR
puts Foo::BAZ
puts Foo::BIN

10
100
1000


## Summary
| Variable Type | Availability |
| --- | --- |
| local_variable = 1 | not available in any other scope |
| @instance_varable = 2 | available within methods of the instance |
| @@class_variable =  3 | available to instances of the class
| @global_variable = 4 | available everywhere |
| CONSTANT = 5 | available everywhere |
| class Klass...end | available everywhere |
| module Mod...end | available everywhere|